In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
#%load_ext cuml.accel
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [3]:
df=pd.read_csv("/kaggle/input/titanic/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
X= df.drop(columns=["PassengerId","Name","Ticket"])
y=df["Survived"]

In [5]:
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # handle NaNs
    ('encoder', OneHotEncoder(handle_unknown='ignore'))    # encode after imputin
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # fill NaNs
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ('cat', cat_pipeline, ["Sex","Cabin","Embarked"]),
    ("num",num_pipeline,["Pclass","Age","SibSp","Parch","Fare"])
])

In [6]:
rf = RandomForestClassifier()
cat = CatBoostClassifier(verbose=False)
lg = LogisticRegression()


# Combine with Voting
voting_clf = VotingClassifier(
    estimators=[
         ('rf', rf),
        ('cat', cat),
        ('lg', lg),
    ],
    voting='soft'  # or 'hard'
)

# Final pipeline with preprocessing + ensemble
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', voting_clf)
])

scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
print(f'CV Accuracy: {scores.mean():.3f} ± {scores.std():.3f}')

CV Accuracy: 0.826 ± 0.019


In [7]:
best_estimator=pipe.fit(X,y)

+++++++++++++++++++++++++++++++++TEST++++++++++++++++++++++++++++++++++++++++

In [8]:
df = pd.read_csv("/kaggle/input/titanic/test.csv")

In [9]:
X= df.drop(columns=["PassengerId","Name","Ticket"])

In [10]:
predictions = best_estimator.predict(X)

In [11]:
submission = pd.DataFrame({
    "PassengerId": df["PassengerId"],
    "Transported": predictions  # Make sure it's True/False not 1/0
})

submission.to_csv("submission.csv", index=False)
print("📁 Saved submission.csv for Kaggle")

📁 Saved submission.csv for Kaggle
